In [62]:
import os

# dataset from https://www.kaggle.com/datasets/elvinagammed/chatbots-intent-recognition-dataset

HERE = os.getcwd()
INTENT_DATA = "./Intent.json"


In [ ]:
!pip install nltk pandas numpy torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.8 kB 660.6 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.8 kB 525.1 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 544.1 kB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     ---------------------------- --------- 30.7/41.5 kB 186.2 kB/s eta 0:00:01
     ---------------------------- --------- 30.7/41.5 kB 186.2 kB/s eta 0:00:01
     ---------------------------- --------- 30.7/41.5 kB 186.2 kB/s eta 0:00:01
     ---------------------------- --------- 30.7/41.5 kB 186.2 kB/s eta 0:00:01
     ---------------------------- --------- 30.7/41.5 kB 186.2 kB/s eta 0:00:01
     --

In [63]:
import nltk
import numpy as np
from nltk.stem.porter import PorterStemmer

nltk.download('punkt_tab')

steamer = PorterStemmer()

def tokenize(sentence):
  return nltk.word_tokenize(sentence)

def stem(word):
  """
  returns the stemmed version of a word
  """
  return steamer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
  """
  returns a bag of words array

  sentence = ["hello", "how", "are", "you"]
  words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
  bag = [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
  """
  tokenized_sentence = [stem(w) for w in tokenized_sentence]
  bag = np.zeros(len(all_words), dtype=np.float32)
  for idx, w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx] = 1.0
  return bag



[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kubus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [65]:
import json

with open(INTENT_DATA, "r") as f:
  intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents["intents"]:
  tag = intent["intent"].lower()
  tags.append(tag)
  for pattern in intent["text"]:
    w = tokenize(pattern)
    all_words.extend(w)
    xy.append((w, tag))

ignore_words = ["?", "!", ".", ","]
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(tags)
print(all_words)

['clever', 'courtesygoodbye', 'courtesygreeting', 'courtesygreetingresponse', 'goodbye', 'gossip', 'greeting', 'greetingresponse', 'jokes', 'namequery', 'nottalking2u', 'podbaydoor', 'podbaydoorresponse', 'realnamequery', 'selfaware', 'shutup', 'swearing', 'thanks', 'timequery', 'understandquery', 'whoami']
["'s", 'a', 'about', 'adam', 'adio', 'am', 'ani', 'anyon', 'are', 'awar', 'bay', 'be', 'bella', 'bore', 'bye', 'call', 'camera', 'can', 'cheer', 'clever', 'commun', 'comprendo', 'consciou', 'could', 'do', 'door', 'enough', 'for', 'friend', 'fuck', 'geniou', 'get', 'girl', 'give', 'good', 'goodby', 'gossip', 'got', 'great', 'have', 'hear', 'hello', 'help', 'hi', 'hola', 'hope', 'how', 'hya', 'i', 'identifi', 'in', 'intellig', 'is', 'it', 'joke', 'know', 'later', 'laugh', 'make', 'me', 'mean', 'meant', 'more', 'my', "n't", 'name', 'need', 'not', 'off', 'ok', 'open', 'pleas', 'pod', 'prove', 'quiet', 'real', 'say', 'see', 'self', 'self-awar', 'shhh', 'shit', 'shut', 'some', 'speak', 's

In [66]:
import numpy as np

X_train, y_train = [], []

for (pattern_sentence, tag) in xy:
  bag = bag_of_words(pattern_sentence, all_words)
  X_train.append(bag)

  label = tags.index(tag)
  y_train.append(label) # for CrossEntropyLoss, the one hot cannot be used.

X_train = np.array(X_train)
y_train = np.array(y_train)

In [67]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class ChatDataset(Dataset):
  def __init__(self, x_train, y_train):
    self.train_size = len(x_train)
    self.x_data = x_train
    self.y_data = y_train

  def __len__(self):
    return self.train_size

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

In [70]:
batch_size = 16

dataset = ChatDataset(X_train, y_train)
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [71]:
# feed forward neural net

class NeuralNetModule(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNetModule, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, num_classes)

    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    out = self.relu(out)
    out = self.l3(out)

    # no activation for softmax!!!!!!!!!!
    return out

In [72]:
input_size = len(X_train[0])
hidden_size = 128
output_size = len(tags)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetModule(input_size, hidden_size, output_size).to(device)


In [73]:
learning_rate = 0.001
num_epochs = 1000

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [74]:
for epoch in range(num_epochs):
  for (words, labels) in data_loader:
    words = words.to(device)
    labels = labels.to(device)

    # forward
    outputs = model(words)
    loss = criterion(outputs, labels)

    # backward and optimizer
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch + 1) % 100 == 0:
    print(f"epoch {epoch + 1}/{num_epochs}, loss={loss.item():.4f}")

print(f"final loss, loss={loss.item():.4f}")

epoch 100/1000, loss=0.0017
epoch 200/1000, loss=0.0002
epoch 300/1000, loss=0.0004
epoch 400/1000, loss=0.0001
epoch 500/1000, loss=0.0000
epoch 600/1000, loss=0.0000
epoch 700/1000, loss=0.0000
epoch 800/1000, loss=0.0000
epoch 900/1000, loss=0.0000
epoch 1000/1000, loss=0.0000
final loss, loss=0.0000


In [75]:
model_data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

torch.save(model_data, "model_data.pth")


In [76]:
loaded_model_data = torch.load("model_data.pth")

model_state = loaded_model_data["model_state"]
input_size = loaded_model_data["input_size"]
output_size = loaded_model_data["output_size"]
hidden_size = loaded_model_data["hidden_size"]
all_words = loaded_model_data["all_words"]
tags = loaded_model_data["tags"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetModule(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()


NeuralNetModule(
  (l1): Linear(in_features=111, out_features=128, bias=True)
  (l2): Linear(in_features=128, out_features=128, bias=True)
  (l3): Linear(in_features=128, out_features=21, bias=True)
  (relu): ReLU()
)

In [77]:
import re

def extract_name(text: str) -> str:
    match = re.search(r"(my name is|i am|i'm)\s+([A-Z][a-z]+)", text, re.IGNORECASE)
    if match:
        return match.group(2)
    return "human"

In [80]:
import random

def create_bot_response(user_input: str):
    sentence = tokenize(user_input)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    print(f"==== prob {prob.item()} ======\n")
    print(f"User said: {user_input}\n")

    if prob.item() >= 0.75:
        for intent in intents["intents"]:
            if tag == intent["intent"].lower():
                response = random.choice(intent["responses"])
                if "<NAME>" in response:
                    name = extract_name(user_input)
                    response = response.replace("<NAME>", name)
                print(f"AI: {response}")
    else:
        print("I'm sorry but I don't understand 😔")
    print("\n------------------------------------------------\n")


In [86]:
sentences = [
    "Damn, I am so so so bored!",
    "I am Wiktor",
    "Hi",
    "Hmm, I feel so lazy, open the window for me :3",\
    "Tell me something funny :))",
    "Do you understand what I am saying"
]

for s in sentences:
    create_bot_response(s)

==== prob 0.9685189723968506 ======

User said: Damn, I am so so so bored!

AI: Good! Hi so, how can I help you?

------------------------------------------------

==== prob 0.9734281897544861 ======

User said: I am Wiktor

AI: OK! Hola Wiktor, how can I help you?

------------------------------------------------

==== prob 0.9999951124191284 ======

User said: Hi

AI: Hi there, what can I do for you? ðŸ˜Š

------------------------------------------------

==== prob 0.9778328537940979 ======

User said: Hmm, I feel so lazy, open the window for me :3

AI: Let me see

------------------------------------------------

==== prob 0.30210885405540466 ======

User said: Tell me something funny :))

I'm sorry but I don't understand 😔

------------------------------------------------

==== prob 1.0 ======

User said: Do you understand what I am saying

AI: I read you loud and clear!

------------------------------------------------

